In [1]:
import sys
sys.path = ['D:/projects/cleanX'] + sys.path

In [2]:
from cleanX.dataset_processing import MLSetup

In [3]:
setup = MLSetup('example_csv.csv', 'example_csv.csv')
report = setup.generate_report()
report.to_ipwidget()

,Unnamed: 0,Unnamed: 0.1_x,X_ray_image_name_x,Label_x,Dataset_type_x,Label_2_Virus_category_x,Label_1_Virus_category_x,gender_x,Unnamed: 0.1_y,X_ray_image_name_y,Label_y,Dataset_type_y,Label_2_Virus_category_y,Label_1_Virus_category_y,gender_y
0,0,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN,Male,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN,Male
1,1,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN,Female,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN,Female
2,2,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN,Female,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN,Female
3,3,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN,Male,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN,Male
4,4,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN,Female,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5905,5905,5928,person1637_virus_2834.jpeg,Pnemonia,TEST,NaN,Virus,Male,5928,person1637_virus_2834.jpeg,Pnemonia,TEST,NaN,Virus,Male
5906,5906,5929,person1635_virus_2831.jpeg,Pnemonia,TEST,NaN,Virus,Female,5929,person1635_virus_2831.jpeg,Pnemonia,TEST,NaN,Virus,Female
5907,5907,5930,person1634_virus_2830.jpeg,Pnemonia,TEST,NaN,Virus,Female,5930,person1634_virus_2830.jpeg,Pnemonia,TEST,NaN,Virus,Female
5908,5908,5931,person1633_virus_2829.jpeg,Pnemonia,TEST,NaN,Virus,Male,5931,person1633_virus_2829.jpeg,Pnemonia,TEST,NaN,Virus,Male
